In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import sklearn.metrics as skm
import sklearn.model_selection as skms

In [111]:
pd.options.display.float_format = '{:.5f}'.format

In [112]:
df = pd.read_csv(r'CSVs\Combined_CSV.csv')
df.head()

,Year,DCSP,TEMP_CHG,ANN_PRCP,FREQ_EXT_EVE,AG_TFP_IND,GDP,POP_GRW,C_Yld,RFWC,CH4_EMSN
0,1990,2201.04540,-0.09800,1431.01490,14,68.60630,368.74976,2.19900,1.89120,1661.20560,70.82000
1,1991,2290.30030,0.17900,1134.92300,11,68.64989,303.85044,2.13700,1.92630,1626.65320,70.42000
2,1992,2323.17000,0.01700,1000.06630,8,70.08235,317.55873,2.12300,2.02480,1593.25840,70.16000
3,1993,2256.15700,0.21100,1174.99670,12,70.40960,301.50079,2.07700,2.08490,1560.96290,70.61000
4,1994,2275.49020,0.10000,1275.73010,6,71.16675,346.22739,2.01200,2.11550,1529.73470,70.31000


In [113]:
df = df.drop('Year', axis=1)
df.head()

,DCSP,TEMP_CHG,ANN_PRCP,FREQ_EXT_EVE,AG_TFP_IND,GDP,POP_GRW,C_Yld,RFWC,CH4_EMSN
0,2201.04540,-0.09800,1431.01490,14,68.60630,368.74976,2.19900,1.89120,1661.20560,70.82000
1,2290.30030,0.17900,1134.92300,11,68.64989,303.85044,2.13700,1.92630,1626.65320,70.42000
2,2323.17000,0.01700,1000.06630,8,70.08235,317.55873,2.12300,2.02480,1593.25840,70.16000
3,2256.15700,0.21100,1174.99670,12,70.40960,301.50079,2.07700,2.08490,1560.96290,70.61000
4,2275.49020,0.10000,1275.73010,6,71.16675,346.22739,2.01200,2.11550,1529.73470,70.31000


In [114]:
def splitting_data(X:pd.DataFrame, Y:pd.DataFrame, test_size:float, seed:int):
    np.random.seed(seed)

    X_test_index = np.random.choice(X.index, int(test_size*len(X.index)), False)
    Y_test_index = np.random.choice(Y.index, int(test_size*len(Y.index)), False)

    X_train = X.drop(X_test_index)
    Y_train = Y.drop(Y_test_index, axis=0)

    X_test = X.loc[X_test_index, :]
    Y_test = Y.loc[Y_test_index, :]

    X_train, Y_train, X_test, Y_test = X_train.reset_index(drop=True), Y_train.reset_index(drop=True), X_test.reset_index(drop=True), Y_test.reset_index(drop=True)

    return {'X_train':X_train, 
            'Y_train':Y_train, 
            'X_test':X_test, 
            'Y_test':Y_test}

In [115]:
def prepare_data(df:pd.DataFrame, y_var_col:str, x_var_cols:list[str], test_size:float, seed:int, z_normalise:bool):
    if z_normalise:
        df = (df - df.mean())/(df.std())
    
    y_var = df[y_var_col]
    x_vars = sm.add_constant(df[x_var_cols])

    if isinstance(y_var, pd.Series):
        y_var = y_var.to_frame()
    
    if isinstance(x_vars, pd.Series):
        x_vars = x_vars.to_frame()

    prepped_data_dict = splitting_data(X=x_vars, Y=y_var, test_size=test_size, seed=seed)

    return prepped_data_dict

In [116]:
def isolate_vars(df:pd.DataFrame):
    split_dict = df.to_dict(orient='series')

    return split_dict

In [117]:
data_dict = prepare_data(df, 'DCSP', df.columns.values[1:], 0.3, 100, True)
x_var_dict = isolate_vars(data_dict['X_train'])

# Intial Model

### Y vs All X

In [118]:
ini_mod = sm.OLS(data_dict['Y_train'], data_dict['X_train'])
ini_mod_res = ini_mod.fit()
print(ini_mod_res.summary())
pvalue_table = ini_mod_res.pvalues.to_frame().rename(columns={0:'P_Vals'})
display(pvalue_table)

                            OLS Regression Results                            
Dep. Variable:                   DCSP   R-squared:                       0.907
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     13.06
Date:                Tue, 04 Feb 2025   Prob (F-statistic):           6.40e-05
Time:                        21:08:45   Log-Likelihood:                0.49359
No. Observations:                  22   AIC:                             19.01
Df Residuals:                      12   BIC:                             29.92
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.2659      0.072     -3.668   

,P_Vals
const,0.00322
TEMP_CHG,0.46394
ANN_PRCP,0.02386
FREQ_EXT_EVE,0.85295
AG_TFP_IND,0.02301
GDP,0.01613
POP_GRW,0.13708
C_Yld,0.00156
RFWC,0.46033
CH4_EMSN,0.00707


In [119]:
alpha = 0.05
pvalue_table['< alpha'] = pvalue_table['P_Vals'] < alpha
display(pvalue_table)

,P_Vals,< alpha
const,0.00322,True
TEMP_CHG,0.46394,False
ANN_PRCP,0.02386,True
FREQ_EXT_EVE,0.85295,False
AG_TFP_IND,0.02301,True
GDP,0.01613,True
POP_GRW,0.13708,False
C_Yld,0.00156,True
RFWC,0.46033,False
CH4_EMSN,0.00707,True


### Y vs Each X variable

In [120]:
def y_vs_each_x(data_dict, x_var_dict):
    rsq_dict = {'R_sq_Vals': [sm.OLS(data_dict['Y_train'], sm.add_constant(x_var_dict[x])).fit().rsquared for x in x_var_dict]}
    rsq_table = pd.DataFrame.from_dict(rsq_dict)
    rsq_table.index = list(x_var_dict.keys())
    return rsq_table

In [121]:
rsq_table = y_vs_each_x(data_dict, x_var_dict)
display(rsq_table)

,R_sq_Vals
const,0.00000
TEMP_CHG,0.14452
ANN_PRCP,0.04735
FREQ_EXT_EVE,0.01027
AG_TFP_IND,0.63500
GDP,0.59246
POP_GRW,0.54566
C_Yld,0.68407
RFWC,0.46490
CH4_EMSN,0.35212


### Conclusion from Initial Stage

In [122]:
table1 = pd.concat([pvalue_table, rsq_table], axis=1)
view = table1.sort_values(by='< alpha')
display(view)

,P_Vals,< alpha,R_sq_Vals
TEMP_CHG,0.46394,False,0.14452
FREQ_EXT_EVE,0.85295,False,0.01027
POP_GRW,0.13708,False,0.54566
RFWC,0.46033,False,0.46490
const,0.00322,True,0.00000
ANN_PRCP,0.02386,True,0.04735
AG_TFP_IND,0.02301,True,0.63500
GDP,0.01613,True,0.59246
C_Yld,0.00156,True,0.68407
CH4_EMSN,0.00707,True,0.35212


- Taking a look at the above table, it can be seen that the first 4 predictors all have p-values greater than our significance level.

- However, if we look at ANN_PRCP (annual precipitation levels), we can see that although it's p-value is lesser than alpha, indicating that it is a significant contributer, it's R_square value is only 0.04735 with DCSP.

- Therefore, at this stage, we cannot say if these predictors will be significant in the final model. Instead, the only predictor we can comfortably remove at this stage is FREQ_EXT_EVE (Frequency of extreme weather events) as it has the highest p-value and the lowest R-square value with DCSP.

- TEMP_CHG, POP_GRW, RFWC will all still be kept at this stage, as despite having p-values that that are higher than alpha, they still have R_square values greater than ANN_PRCP, making it unclear if they will turn out to be siginificant contributers later on.

# Backward Selection

- Using the backward selection method, after every model is constructed, we remove the predictor with the highest p-value, and then build the model again. We repeat this process until the remaining predictors have p-value lesser than 0.05.

- In a multiple regression model, each predictor uses the information from the other predictors to contribute towards the prediction of the response variable causing each p-value to be adjusted for other predictors in the model. 

- Therefore, by rebuilding the model again and again after each removal, we ensure that we are truly removing the predictors that are not significant.

In [123]:
X_train_v2 = data_dict['X_train'].drop('FREQ_EXT_EVE', axis=1)
X_train_v2.head()

,const,TEMP_CHG,ANN_PRCP,AG_TFP_IND,GDP,POP_GRW,C_Yld,RFWC,CH4_EMSN
0,1.00000,-1.84096,2.28328,-1.00965,-0.95098,1.57526,-1.51168,1.97661,1.61646
1,1.00000,-1.49988,-1.85118,-0.91450,-1.03560,1.37191,-1.20345,1.61719,1.37703
2,1.00000,-0.92450,-0.17292,-0.89341,-1.06214,1.24883,-1.06479,1.44636,1.54028
3,1.00000,-1.25371,0.79350,-0.84461,-0.98821,1.07491,-0.99420,1.28118,1.43145
4,1.00000,-1.82316,0.12746,-0.81938,-0.87470,0.91170,-0.73326,0.82043,0.83287


In [124]:
def Backward_Selection(Y_train, X_train):
    model = sm.OLS(Y_train, X_train)
    result = model.fit()
    pvalue_table = result.pvalues.to_frame().rename(columns={0:'P_Vals'})
    #print(f'This is iteration 0')
    #print(pvalue_table)

    counter = 1
    while any(pval > alpha for pval in pvalue_table['P_Vals'].to_list()):
        #print(f'This is iteration {counter}')
        var_to_rem = [pvalue_table.index[x] for x,y in zip(*np.where(pvalue_table.values == pvalue_table['P_Vals'].max()))][0]
        #print(f'Var to remove: {var_to_rem}')
        if var_to_rem == 'const':
            pvalue_table = pvalue_table.drop('const')
            if pvalue_table['P_Vals'].max() > alpha:
                var_to_rem = [pvalue_table.index[x] for x,y in zip(*np.where(pvalue_table.values == pvalue_table['P_Vals'].max() and pvalue_table.values > 0.05))][0]
            else:
                break
        X_train = X_train.drop(var_to_rem, axis=1)
        #display(X_train.head(3))
        model = sm.OLS(Y_train, X_train)
        result = model.fit()
        pvalue_table = result.pvalues.to_frame().rename(columns={0:'P_Vals'})
        #print(pvalue_table)
        counter += 1

    output_dict = {'Model': model,
                   'Result': result,
                   'P_Values': pvalue_table,
                   'new_X_train': X_train,
                   'Final_Vars': X_train.columns.to_list()}
    
    return output_dict

In [125]:
final_model = Backward_Selection(data_dict['Y_train'], X_train_v2)
#print(final_model)

# Analysis of Model

In [127]:
print(final_model['Result'].summary())

                            OLS Regression Results                            
Dep. Variable:                   DCSP   R-squared:                       0.882
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     23.81
Date:                Tue, 04 Feb 2025   Prob (F-statistic):           6.95e-07
Time:                        21:08:45   Log-Likelihood:                -2.2146
No. Observations:                  22   AIC:                             16.43
Df Residuals:                      16   BIC:                             22.98
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2602      0.070     -3.717      0.0

## Real World Implications

In [128]:
final_model['Result'].params

const        -0.26023
ANN_PRCP     -0.22083
AG_TFP_IND   -3.16764
GDP           3.01611
C_Yld         2.29334
CH4_EMSN      1.56195
dtype: float64

- From the coefficients, we can draw conclusions regarding the predictors and how they affect Daily Supply of Calories per Person (DSCP) and hence, food security in India. 

- Firstly, Annual Precipitation levels (ANN_PRCP) have a negative coefficient, which signifies that as annual precipitation levels increase, DSCP decreases. 
    - In the real world, this could possibly be explained as higher levels of rainfall contributing to floods and other events which ultimately results in less food being available/harvested for the population.

- The magnitude of the coefficients for Crop yields of Cereals (C_Yld) and GDP are are very high, possibly indicating that this is a strong indicators of DSCP which would be within expectations. 

- However, Share of CH4 emissions (CH4_EMSN) has a much larger coefficient than both Annual Precipitation levels (ANN_PRCP) and GDP per Capita (GDP). Furthermore, this coefficient is positive, indicating that an increase in emissions contributes to an increase in DSPC.

- One possible explanation for this unexpected conclusion could be that an increase in emissions indicates an increase in use of resources and agricultural productivity, particularly increase in rice cultivation, as the decay of organic matter in rice paddies contribute to methane emissions.

- If we were to follow the model and recommend the increase of emissions and practices that contribute to the emissions, we would be putting the sustainability of resources and ecosystems at risk at the possible of short-term increases in DSPC.

- Therefore, this exposes a real-world weakness of the model. The model analyses trends of the past 31 years but is unable to account for the impact of these predictors far into the future or such nuances as presented above.

- Another unexpected conclusion is that the coefficient for Agricultural Productivity Index (AG_TFP_IND) the magnitude of the coefficient is high but it's sign is negative, indicating that as the Productivity Index increases, DCSP decreases, which would be against expectiations.

- One possible explanation is that lower caloric availability leads to policy changes that boost agricultural efficiency (rather than TFP increasing calorie supply), and hence, the negative sign might reflect this reversed causal relationship.

- However, there also exists the possibility that the model might have attributed it's effect incorrectly due to high levels of multi-collinearity. In the following sections, we will conduct the appropritate tests.

## Evaluation through metrics

In [130]:
print(final_model['Result'].rsquared_adj)

0.8444893537662188


In [132]:
X_test = data_dict['X_test']
X_test_v2 = X_test[final_model['Final_Vars']]
display(X_test_v2)

,const,ANN_PRCP,AG_TFP_IND,GDP,C_Yld,CH4_EMSN
0,1.00000,0.31404,-0.62659,-0.66156,-0.33920,0.03113
1,1.00000,0.12569,1.62344,1.67597,1.34108,-1.06446
2,1.00000,-0.55738,-1.00684,-1.05826,-1.43070,1.47135
3,1.00000,-0.31675,1.01390,1.06801,0.70962,-0.91209
4,1.00000,-0.06679,-0.82646,-0.94292,-1.00296,1.15936
5,1.00000,0.65636,0.06888,0.08172,0.21105,-0.39332
6,1.00000,-0.79617,1.92727,1.70309,1.68808,-1.24222
7,1.00000,0.97782,0.10988,0.13004,0.08508,-0.21556
8,1.00000,-0.01533,-0.77043,-0.90002,-0.84308,1.10132


In [148]:
y_pred = final_model['Result'].predict(X_test_v2).to_frame()
display(y_pred)

,0
0,-1.06935
1,1.03736
2,-1.12260
3,0.02204
4,-0.96077
5,-0.50721
6,0.87844
7,-0.57359
8,-0.74424


In [149]:
rmse = np.sqrt( np.mean( (y_pred.to_numpy() - data_dict['Y_test'].to_numpy()) ** 2 ) )
print(rmse, '\n')
print(data_dict['Y_test'].mean(), '\n')
print(data_dict['Y_test'].std())

1.5526558717183112 

DCSP   0.55859
dtype: float64 

DCSP   1.26228
dtype: float64


In [142]:
print(final_model['Result'].f_pvalue)

6.950188194646521e-07


- The adjusted R_square metric indicates the extent to which the predictors can explain the response variable. It is a better metric to evaluate a multiple linear regression model as it adjusts for additional predictors since the R_square value always increases the more predictors that are added. The adjusted R_square for our final model is 0.844 (3.s.f), implying that the variables are able to account for a good proportion of the y_variable

- For the whole model test, we need to look at the Significance F value, which serves as the p-value for the whole model test. It has a value of 5.2087E-12, implying that at least 1 variable in our model is significant. 

- Following this, looking at the p-values of all the predictors in the final model (as presented in the previous section), we can see that the partial tests reveal that all our variables are significant and contribute to the final model.

- Therefore, when analysing through these metrics, we can see that this final model is able to explain a good proportion of the response variable with predictors that are significant.

- However, taking a look at the RMSE value, it is both higher than the mean and standard deviation, although of a comparable value to the standard deviation.

- This could indicate that the model is not entirely accurate. The reasons for this could lie in wether the assumptions of linear regression are met by this model or not.
